In [1]:
# Importing and mounting Google Drive to access and store datasets directly from Google Colab.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#Importing the necessary packages such as pandas, string, nltk, counter and stopwords
import pandas as pd
import string
from collections import Counter
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

**Data set exploration**

The initial step starts with loading the csv data file and making it ready for the further exploration for the machine learning technique.
The data loading helps us to understand the structure of the dataset and very important step before proceeding the technique.

In [3]:
#Loading the data
data= pd.read_csv('/content/drive/MyDrive/76.csv')


In [4]:
#Head command dispays the first few rows and columns for checking the data
data.head()

,Unnamed: 0,category,headline,authors,link,short_description,date
0,0,QUEER VOICES,Gwist Recap: The Best Video Clips From The Gay...,NaN,https://www.huffingtonpost.com/entry/best-gay-...,Gwist is a YouTube channel that brings togethe...,2013-11-30
1,1,QUEER VOICES,"For Transgender Service Members, Donald Trump'...",Andy Campbell,https://www.huffingtonpost.com/entry/for-trans...,"""I fought for America… Why not fight for me?” ...",2017-07-26
2,2,QUEER VOICES,'Transparent' Co-Producer On Why The Show Prio...,Kira Brekke,https://www.huffingtonpost.com/entry/transpare...,"""The future of transgender representation is r...",2015-10-05
3,3,QUEER VOICES,PHOTOS: 'Orange Is The New Black' Celebrates G...,Alana Horowitz Satlin,https://www.huffingtonpost.com/entry/ointb-gay...,NaN,2014-06-29
4,4,QUEER VOICES,These Absurd Lawsuits Show Why The Anti-Gay Mo...,Lila Shapiro,https://www.huffingtonpost.com/entry/suing-hom...,NaN,2015-05-09


In [5]:
#Describe command is used to display the summary statistics of the the given data
data.describe()

,Unnamed: 0
count,8000.00000
mean,3999.50000
std,2309.54541
min,0.00000
25%,1999.75000
50%,3999.50000
75%,5999.25000
max,7999.00000


Preprocess_text is a function that transforms a given text into words, lowercase it, and removes punctuation. After that, it eliminates stopwords often used in English from the tokens and returns the cleaned word list.

In [6]:
# Using Function to preprocess text

def preprocess_text(text):
    text = text.lower()  # Convert to lowercase
    text = text.translate(str.maketrans('', '', string.punctuation))  # Remove punctuation
    tokens = text.split()  # Tokenize
    tokens = [word for word in tokens if word not in stopwords.words('english')]  # Remove stopwords
    return tokens

The given code applies a preprocessing function after eliminating any missing values from the text in the dataset's headline and short_description columns. Entries from these columns are combined for each category after the data is grouped by categories. It uses these tokens to identify and store the most frequently used terms for headlines and brief descriptions in a dictionary called common_terms_by_category, which associates the most frequently used terms with each category.

In [7]:
#PREPROCESSING FUNCTION

# Preprocessing the headline and short_description columns
data['headline_tokens'] = data['headline'].dropna().apply(preprocess_text)
data['short_description_tokens'] = data['short_description'].dropna().apply(preprocess_text)

# Applying Function to get the most common terms
def get_most_common_terms(tokens, num_terms=10):
    counter = Counter(tokens)
    return counter.most_common(num_terms)

# Analyze the most common terms for each category
category_groups = data.groupby('category')
common_terms_by_category = {}

for category, group in category_groups:
    all_headline_tokens = [token for tokens in group['headline_tokens'].dropna() for token in tokens]
    all_description_tokens = [token for tokens in group['short_description_tokens'].dropna() for token in tokens]
    common_terms_by_category[category] = {
        'headline': get_most_common_terms(all_headline_tokens),
        'short_description': get_most_common_terms(all_description_tokens)
    }
common_terms_by_category

{'QUEER VOICES': {'headline': [('gay', 1250),
   ('new', 413),
   ('queer', 388),
   ('lgbt', 355),
   ('trans', 346),
   ('transgender', 288),
   ('lgbtq', 282),
   ('marriage', 266),
   ('people', 219),
   ('video', 206)],
  'short_description': [('gay', 648),
   ('people', 443),
   ('new', 344),
   ('one', 341),
   ('lgbt', 338),
   ('us', 287),
   ('week', 282),
   ('community', 259),
   ('like', 250),
   ('transgender', 236)]},
 'RELIGION': {'headline': [('pope', 189),
   ('francis', 125),
   ('meditation', 103),
   ('daily', 101),
   ('church', 93),
   ('muslim', 79),
   ('faith', 69),
   ('religious', 66),
   ('new', 61),
   ('muslims', 53)],
  'short_description': [('people', 148),
   ('us', 125),
   ('one', 119),
   ('need', 100),
   ('god', 95),
   ('time', 89),
   ('spiritual', 88),
   ('church', 86),
   ('help', 82),
   ('religious', 80)]}}

In [8]:

# Function for calculating the length of text
def text_length(text):
    if pd.isna(text):
        return None
    return len(text.split())


# Calculating the length of headlines and short descriptions, handling NaN values
data['headline_length'] = data['headline'].apply(text_length)
data['short_description_length'] = data['short_description'].apply(text_length)

# Grouping by category and calculate average lengths
average_lengths = data.groupby('category').agg({
    'headline_length': 'mean',
    'short_description_length': 'mean'
}).reset_index()

# Displaying the dataframe
print(average_lengths)

       category  headline_length  short_description_length
0  QUEER VOICES        10.358519                 22.057795
1      RELIGION         8.716867                 25.580913


**This step is very crucial to check the missing values in the certain columns and rows**

In [9]:

# Checking for the missing values
missing_values = data.isnull().sum()

# Displaying the missing values
missing_values



Unnamed: 0                     0
category                      20
headline                      19
authors                     1335
link                          13
short_description           1278
date                          14
headline_tokens               19
short_description_tokens    1278
headline_length               19
short_description_length    1278
dtype: int64

**OBSERVATIONS**

The frequently used terms in descriptions and headlines vary greatly depending on the category, which reflects the variety of subjects and themes covered.
Compared to articles in the "RELIGION" category, those under the "QUEER VOICES" category typically have slightly longer headlines but shorter descriptions.
While the category, title, and date columns have relatively few missing values, the authors and short_description columns have a significant amount of missing entries.

The 'category' or 'headline' columns include missing values, which the script removes to clean up the dataset, a step that is essential for training the model. The cleaned data is then divided using a divided approach based on the category into training, validation, and test sets, ensuring representative distribution across all sets. Finally, it saves these data splits to distinct CSV files so that they can be loaded individually at different stages of the building of a machine learning model.

In [10]:

from sklearn.model_selection import train_test_split

# Removing rows with missing category or headline values, as these are critical for modeling
cleaned_data = data.dropna(subset=['category', 'headline'])

# Splitting the data into training, validation, and test sets
train_data, temp_data = train_test_split(cleaned_data, test_size=0.4, random_state=42, stratify=cleaned_data['category'])
valid_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42, stratify=temp_data['category'])

# Saving the splits into separate CSV files
train_csv_path = '/content/sample_data/train.csv'
valid_csv_path = '/content/sample_data/valid.csv'
test_csv_path = '/content/sample_data/test.csv'

train_data.to_csv(train_csv_path, index=False)
valid_data.to_csv(valid_csv_path, index=False)
test_data.to_csv(test_csv_path, index=False)

(train_data.shape, valid_data.shape, test_data.shape), train_csv_path, valid_csv_path, test_csv_path



(((4776, 11), (1592, 11), (1593, 11)),
 '/content/sample_data/train.csv',
 '/content/sample_data/valid.csv',
 '/content/sample_data/test.csv')

To maintain category consistency throughout the splits, stratified sampling is used to make sure that the ratio of categories in each split closely resembles the distribution found in the original dataset. The data is separated into three parts: a training set, which makes up 60% of the data and offers a significant amount of information for efficient model training; a validation set, which makes up 20% of the data and is used to select the best model and adjust parameters; and a test set, which is also 20% of the data but is only used for the final evaluation and remains secret throughout the training and validation phases to ensure the quality of the data.


In [11]:

import pandas as pd
from sklearn.model_selection import train_test_split
import string
from sklearn.feature_extraction.text import TfidfVectorizer

# Loading the dataset

train_data = pd.read_csv('/content/sample_data/train.csv')
valid_data= pd.read_csv('/content/sample_data/valid.csv')

# Removing rows with missing category or headline values
cleaned_data = data.dropna(subset=['category', 'headline'])

# Filling the missing values in the 'short_description' column with empty strings
cleaned_data['short_description'] = cleaned_data['short_description'].fillna('')

# Splitting the data into training, validation sets
train_data, valid_data = train_test_split(cleaned_data, test_size=0.4, random_state=42, stratify=cleaned_data['category'])

# Defining the text preprocessing function
def preprocess_text(text):
    text = text.lower()  # Convert to lowercase
    text = text.translate(str.maketrans('', '', string.punctuation))  # Remove punctuation
    return text

# Applying the preprocessing to the dataset
train_data['headline'] = train_data['headline'].apply(preprocess_text)
train_data['short_description'] = train_data['short_description'].apply(preprocess_text)
valid_data['headline'] = valid_data['headline'].apply(preprocess_text)
valid_data['short_description'] = valid_data['short_description'].apply(preprocess_text)

# Combining headline and short_description for TF-IDF
train_data['combined_text'] = train_data['headline'] + ' ' + train_data['short_description']
valid_data['combined_text'] = valid_data['headline'] + ' ' + valid_data['short_description']

# Initializing the TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer()

# Fit and transforming the combined text for both train and validation sets
tfidf_train_matrix = tfidf_vectorizer.fit_transform(train_data['combined_text'])
tfidf_valid_matrix = tfidf_vectorizer.transform(valid_data['combined_text'])

# Convert TF-IDF matrix to DataFrame for better visualization
tfidf_train_df = pd.DataFrame(tfidf_train_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out())
tfidf_valid_df = pd.DataFrame(tfidf_valid_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

# Saving the processed data for further steps
train_csv_path = '/content/sample_data/train_tfidf.csv'
valid_csv_path = '/content/sample_data/valid_tfidf.csv'

tfidf_train_df.to_csv(train_csv_path, index=False)
tfidf_valid_df.to_csv(valid_csv_path, index=False)

# Output the shapes to verify the transformation
train_shape = tfidf_train_df.shape
valid_shape = tfidf_valid_df.shape

train_shape, valid_shape

<ipython-input-11-25ee0e3f70f4>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_data['short_description'] = cleaned_data['short_description'].fillna('')


((4776, 15096), (3185, 15096))

To develop binary classification models for distinguishing between the two news categories ("QUEER VOICES" and "RELIGION"), we'll employ two classifiers that are typically taught in machine learning lectures: logistic regression (LR) and support vector machine (SVM). These classifiers were chosen based on their effectiveness and accessibility in binary classification problems.

In [12]:
import pandas as pd
from sklearn.model_selection import train_test_split
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report


# Extracting the labels
y_train = train_data['category']
y_valid = valid_data['category']

# Encoding the labels (binary)
y_train = y_train.apply(lambda x: 1 if x == 'QUEER VOICES' else 0)
y_valid = y_valid.apply(lambda x: 1 if x == 'QUEER VOICES' else 0)

# Logistic Regression Classifier
log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(tfidf_train_matrix, y_train)
log_reg_preds = log_reg.predict(tfidf_valid_matrix)

# SVM Classifier
svm = SVC(kernel='linear')
svm.fit(tfidf_train_matrix, y_train)
svm_preds = svm.predict(tfidf_valid_matrix)

# Evaluating the classifiers
def evaluate_model(true_labels, predictions, model_name):
    accuracy = accuracy_score(true_labels, predictions)
    precision = precision_score(true_labels, predictions)
    recall = recall_score(true_labels, predictions)
    f1 = f1_score(true_labels, predictions)
    print(f'{model_name} Performance:')
    print(f'Accuracy: {accuracy:.4f}')
    print(f'Precision: {precision:.4f}')
    print(f'Recall: {recall:.4f}')
    print(f'F1 Score: {f1:.4f}')
    print('Classification Report:')
    print(classification_report(true_labels, predictions))

# Logistic Regression Evaluation
evaluate_model(y_valid, log_reg_preds, 'Logistic Regression')

# SVM Evaluation
evaluate_model(y_valid, svm_preds, 'Support Vector Machine')

Logistic Regression Performance:
Accuracy: 0.8782
Precision: 0.8644
Recall: 0.9933
F1 Score: 0.9244
Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.53      0.69       797
           1       0.86      0.99      0.92      2388

    accuracy                           0.88      3185
   macro avg       0.91      0.76      0.81      3185
weighted avg       0.89      0.88      0.86      3185

Support Vector Machine Performance:
Accuracy: 0.9224
Precision: 0.9210
Recall: 0.9807
F1 Score: 0.9499
Classification Report:
              precision    recall  f1-score   support

           0       0.93      0.75      0.83       797
           1       0.92      0.98      0.95      2388

    accuracy                           0.92      3185
   macro avg       0.92      0.86      0.89      3185
weighted avg       0.92      0.92      0.92      3185



**Logistic Regression with Support Vector Machine (SVM) Options**

Logistic Regression was chosen for its simplicity and efficacy in binary classification settings. The default parameters are used to guarantee that the model is fast and easy to understand, and they provide a reliable benchmark for performance comparisons.
Support Vector Machine (SVM) is chosen because of its ability to handle high-dimensional spaces, which is also useful for binary classification problems. A linear kernel is used to retain simplicity and computational efficiency. While other kernels, such as RBF, may provide better performance, they usually require more thorough parameter tuning to reach ideal results.

Accuracy predicts the model's overall correctness, whereas precision measures the correctness of positive predictions. Recall measures how successfully the model detects all true positives. The F1 Score, which combines accuracy and recall, is especially useful in imbalanced datasets, ensuring a balanced evaluation by taking into account both parameters.

**Deep learning technique with tensor flow**

In [13]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Converting the  labels to categorical (one-hot encoding)
y_train_keras = tf.keras.utils.to_categorical(y_train, num_classes=2)
y_valid_keras = tf.keras.utils.to_categorical(y_valid, num_classes=2)

# Building a simple neural network
model = Sequential()
model.add(Dense(512, input_shape=(tfidf_train_matrix.shape[1],), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

# Compiling the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Training the model
model.fit(tfidf_train_matrix, y_train_keras, epochs=10, batch_size=32, validation_data=(tfidf_valid_matrix, y_valid_keras))

# Evaluating the model
y_valid_preds_keras = model.predict(tfidf_valid_matrix)
y_valid_preds = y_valid_preds_keras.argmax(axis=1)


# Evaluating the deep learning model
evaluate_model(y_valid, y_valid_preds, 'Deep Learning Model')

Epoch 1/10
150/150 [==============================] - 21s 133ms/step - loss: 0.3805 - accuracy: 0.8193 - val_loss: 0.1962 - val_accuracy: 0.9221
Epoch 2/10
150/150 [==============================] - 25s 166ms/step - loss: 0.0910 - accuracy: 0.9680 - val_loss: 0.2128 - val_accuracy: 0.9237
Epoch 3/10
150/150 [==============================] - 20s 133ms/step - loss: 0.0158 - accuracy: 0.9954 - val_loss: 0.2643 - val_accuracy: 0.9149
Epoch 4/10
150/150 [==============================] - 19s 124ms/step - loss: 0.0035 - accuracy: 0.9996 - val_loss: 0.3086 - val_accuracy: 0.9209
Epoch 5/10
150/150 [==============================] - 19s 128ms/step - loss: 8.4888e-04 - accuracy: 1.0000 - val_loss: 0.3230 - val_accuracy: 0.9218
Epoch 6/10
150/150 [==============================] - 22s 148ms/step - loss: 3.8485e-04 - accuracy: 1.0000 - val_loss: 0.3471 - val_accuracy: 0.9224
Epoch 7/10
150/150 [==============================] - 18s 120ms/step - loss: 1.1282e-04 - accuracy: 1.0000 - val_loss: 0.3

The text categorization deep learning model showed encouraging performance, with an F1 score of 95.14%, recall of 96.36%, and validation accuracy of 92.62% with excellent precision (93.96%). These measurements demonstrate strong categorization skills and efficient management of class imbalance. In spite of reaching 100% training accuracy, the model displayed indications of possible overfitting, as demonstrated by growing validation loss over epochs. To balance performance and avoid overfitting, this shows that additional optimization is required, perhaps through the use of more advanced regularization techniques or early halting. With additional tweaks to improve generalizability and retain high accuracy, the model's overall strong performance across key criteria indicates it could be effectively improved and implemented in real-world text classification settings.

**Error analysis is performed to detect the common flaws in the models which was obtained earlier in the analysis.**

In [14]:
# Evaluating the model
y_valid_preds_keras = model.predict(tfidf_valid_matrix)
y_valid_preds_dl = y_valid_preds_keras.argmax(axis=1)

# Performing Error Analysis
def error_analysis(true_labels, predictions, model_name, data):
    errors = true_labels != predictions
    misclassified = data[errors]
    misclassified_indices = data.index[errors]
    print(f'\n{model_name} Misclassified Examples:')
    for index in misclassified_indices:
        row = data.loc[index]
        predicted_label = 'QUEER VOICES' if predictions[list(data.index).index(index)] == 1 else 'RELIGION'
        print(f"Text: {row['combined_text']}\nTrue Label: {row['category']}\nPredicted Label: {predicted_label}\n")

# Logistic Regression Error Analysis
error_analysis(y_valid, log_reg_preds, 'Logistic Regression', valid_data)

# SVM Error Analysis
error_analysis(y_valid, svm_preds, 'Support Vector Machine', valid_data)

# Deep Learning Model Error Analysis
error_analysis(y_valid, y_valid_preds_dl, 'Deep Learning Model', valid_data)


100/100 [==============================] - 1s 5ms/step

Logistic Regression Misclassified Examples:
Text: the prophet of jordans mists the sun comes up slowly on the banks of louisianas bayous lazy mornings mark the arrival of day even as moss covered trees swing sweetly in the hot and steamy breeze
True Label: RELIGION
Predicted Label: QUEER VOICES

Text: searching for transcendence a report for my money i dont think the desire for transcendence is going away but i do wonder if students will bet their lives that the answer for this search lies beyond our world
True Label: RELIGION
Predicted Label: QUEER VOICES

Text: remembering the bittersweet history of fathers day 
True Label: RELIGION
Predicted Label: QUEER VOICES

Text: a ministry on top of the world 
True Label: RELIGION
Predicted Label: QUEER VOICES

Text: who are we to judge 
True Label: RELIGION
Predicted Label: QUEER VOICES

Text: could this bible story allow conservatives to serve samesex couples for years id written off my

**Logistic regression Error analysis**

The error analysis for the Logistic Regression model demonstrates that it commonly misclassifies articles on broad social justice and political themes. Despite being trained in the specified categories, the model appears to struggle with the sophisticated language used in headlines and descriptions, resulting in inaccurate classifications. This suggests that, while Logistic Regression is a strong linear classifier, it may not fully capture the text's complexity and context, resulting in misclassifications, particularly when the language is subtly indicative of a specific category.

**Support Vector Machines (SVM) Error analysis**

The Support Vector Machine model shows similar misclassification patterns to Logistic Regression, frequently misidentifying articles about political activism, social movements, and specific religious issues. This implies that, while SVM is effective in high-dimensional domains and can handle linear separations well, it has difficulty distinguishing contextually rich and overlapping categories. The errors suggest that SVM could benefit from better text representation techniques or more context for better classification performance.

 **Deep Learning Error analysis**

In addition, the Deep Learning model misclassifies articles where there is a large overlap in subjects between the two categories, even with its more sophisticated architecture. Articles addressing activism, more complex religious subjects, and larger societal issues are mistakenly identified. These mistakes demonstrate the model's difficulty extrapolating from the training set to unobserved samples, where context is essential. This implies that in order to lower misclassification rates and boost overall performance, deep learning models—while capable of capturing increasingly intricate patterns, they still need to be carefully tuned and may require access to additional contextual data.

Next step is the Improvization of the models based on the error analysis

In [15]:
import joblib
# Improving the Models
# Improve Logistic Regression by using class weight to handle imbalance
log_reg = LogisticRegression(max_iter=1000, class_weight='balanced')
log_reg.fit(tfidf_train_matrix, y_train)
log_reg_preds = log_reg.predict(tfidf_valid_matrix)
evaluate_model(y_valid, log_reg_preds, 'Improved Logistic Regression')

# Improve SVM by using class weight to handle imbalance
svm = SVC(kernel='linear', class_weight='balanced')
svm.fit(tfidf_train_matrix, y_train)
svm_preds = svm.predict(tfidf_valid_matrix)
evaluate_model(y_valid, svm_preds, 'Improved Support Vector Machine')

# Improve Deep Learning Model
# Adjust the model architecture and hyperparameters if necessary
model = Sequential()
model.add(Dense(512, input_shape=(tfidf_train_matrix.shape[1],), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(tfidf_train_matrix, y_train_keras, epochs=15, batch_size=32, validation_data=(tfidf_valid_matrix, y_valid_keras))
y_valid_preds_keras = model.predict(tfidf_valid_matrix)
y_valid_preds_dl = y_valid_preds_keras.argmax(axis=1)
evaluate_model(y_valid, y_valid_preds_dl, 'Improved Deep Learning Model')

# Saving the improved models
joblib.dump(log_reg, 'logistic_regression_model.pkl')
joblib.dump(svm, 'svm_model.pkl')
model.save('deep_learning_model.h5')

Improved Logistic Regression Performance:
Accuracy: 0.9287
Precision: 0.9511
Recall: 0.9539
F1 Score: 0.9525
Classification Report:
              precision    recall  f1-score   support

           0       0.86      0.85      0.86       797
           1       0.95      0.95      0.95      2388

    accuracy                           0.93      3185
   macro avg       0.91      0.90      0.90      3185
weighted avg       0.93      0.93      0.93      3185

Improved Support Vector Machine Performance:
Accuracy: 0.9300
Precision: 0.9490
Recall: 0.9581
F1 Score: 0.9535
Classification Report:
              precision    recall  f1-score   support

           0       0.87      0.85      0.86       797
           1       0.95      0.96      0.95      2388

    accuracy                           0.93      3185
   macro avg       0.91      0.90      0.91      3185
weighted avg       0.93      0.93      0.93      3185

Epoch 1/15
150/150 [==============================] - 23s 145ms/step - loss: 0.

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


**Improvization**

Machine learning models performed far better after being adjusted to deal with skewed datasets. Significant improvements were observed when 'balanced' class weight adjustments were made to the Logistic Regression and SVM models. Logistic Regression achieved high precision and F1 scores, with an accuracy of 92.87%, while SVM achieved a slightly higher accuracy of 93.00%. The validation loss trends show that the deep learning model had some overfitting, but it still maintained strong precision and recall, with only a slight decrease in accuracy to 92.18%. These findings highlight the significance of tailoring models to individual dataset difficulties, especially when it comes to maximizing accuracy and fairness by balancing class representation.

**The next step is the cross validation and prediction of the model scores**

In [16]:
from sklearn.model_selection import cross_val_score

# Carrying out the Cross-Validation
merged_data = pd.concat([train_data, valid_data])
merged_labels = pd.concat([y_train, y_valid])

tfidf_merged_matrix = tfidf_vectorizer.transform(merged_data['combined_text'])

log_reg_cv_scores = cross_val_score(log_reg, tfidf_merged_matrix, merged_labels, cv=5, scoring='f1')
svm_cv_scores = cross_val_score(svm, tfidf_merged_matrix, merged_labels, cv=5, scoring='f1')

print(f'Logistic Regression Cross-Validation F1 Scores: {log_reg_cv_scores}')
print(f'SVM Cross-Validation F1 Scores: {svm_cv_scores}')


Logistic Regression Cross-Validation F1 Scores: [0.94812315 0.95130143 0.95206056 0.94692502 0.9594312 ]
SVM Cross-Validation F1 Scores: [0.95142379 0.95083333 0.94763301 0.950021   0.95724367]


**The Logistic Regression**

Cross-validation revealed that the Logistic Regression model has a mean F1 score of 95.11%. This shows excellent performance in accurately distinguishing the two news categories while striking a balance between recall and precision.

**Support Vector Machine  (SVM)**

During cross-validation, the SVM model obtained an F1 score that was somewhat higher, 95.14%. When compared to Logistic Regression, SVM appears to be able to handle the dataset's subtleties and complexity with a little bit more consistency and an advantage in performance metrics.


F1 ratings for both models are above 95%, indicating good precision and recall, demonstrating their excellent performance. When tested on the test set, the retrained SVM model should either retain or outperform its validation results, proving its ability to generalize well to new data. However, this little performance advantage translates to a minimal practical application difference, indicating that both models might be applied to the task with good effectiveness.

In [17]:
#Test Set Evaluation
test_data = pd.read_csv('/content/sample_data/test.csv')
test_data = test_data.dropna(subset=['category', 'headline'])
test_data['short_description'] = test_data['short_description'].fillna('')
test_data['headline'] = test_data['headline'].apply(preprocess_text)
test_data['short_description'] = test_data['short_description'].apply(preprocess_text)
test_data['combined_text'] = test_data['headline'] + ' ' + test_data['short_description']
tfidf_test_matrix = tfidf_vectorizer.transform(test_data['combined_text'])

log_reg = joblib.load('logistic_regression_model.pkl')
svm = joblib.load('svm_model.pkl')
model = tf.keras.models.load_model('deep_learning_model.h5')

y_test = test_data['category'].apply(lambda x: 1 if x == 'QUEER VOICES' else 0)

**Retraining with the best Model (SVM)**

In [18]:
# Merging Train and Validation Datasets
merged_data = pd.concat([train_data, valid_data])
merged_labels = pd.concat([y_train, y_valid])

# Retraining the SVM Model
svm_best = SVC(kernel='linear', class_weight='balanced')
tfidf_merged_matrix = tfidf_vectorizer.transform(merged_data['combined_text'])
svm_best.fit(tfidf_merged_matrix, merged_labels)

# Applying the Retrained Model to the Test Set
test_data = pd.read_csv('/content/sample_data/test.csv')
test_data = test_data.dropna(subset=['category', 'headline'])
test_data['short_description'] = test_data['short_description'].fillna('')
test_data['headline'] = test_data['headline'].apply(preprocess_text)
test_data['short_description'] = test_data['short_description'].apply(preprocess_text)
test_data['combined_text'] = test_data['headline'] + ' ' + test_data['short_description']
tfidf_test_matrix = tfidf_vectorizer.transform(test_data['combined_text'])

# Extracting test labels
y_test = test_data['category'].apply(lambda x: 1 if x == 'QUEER VOICES' else 0)

# Predicting using the retrained SVM model
svm_test_preds = svm_best.predict(tfidf_test_matrix)

# Evaluating the model
evaluate_model(y_test, svm_test_preds, 'Retrained SVM')

# Function to evaluate the model
def evaluate_model(true_labels, predictions, model_name):
    accuracy = accuracy_score(true_labels, predictions)
    precision = precision_score(true_labels, predictions)
    recall = recall_score(true_labels, predictions)
    f1 = f1_score(true_labels, predictions)
    print(f'{model_name} Performance:')
    print(f'Accuracy: {accuracy:.4f}')
    print(f'Precision: {precision:.4f}')
    print(f'Recall: {recall:.4f}')
    print(f'F1 Score: {f1:.4f}')
    print('Classification Report:')
    print(classification_report(true_labels, predictions))

# Displaying the results
evaluate_model(y_test, svm_test_preds, 'Retrained SVM')

Retrained SVM Performance:
Accuracy: 0.9856
Precision: 0.9975
Recall: 0.9832
F1 Score: 0.9903
Classification Report:
              precision    recall  f1-score   support

           0       0.95      0.99      0.97       399
           1       1.00      0.98      0.99      1194

    accuracy                           0.99      1593
   macro avg       0.97      0.99      0.98      1593
weighted avg       0.99      0.99      0.99      1593

Retrained SVM Performance:
Accuracy: 0.9856
Precision: 0.9975
Recall: 0.9832
F1 Score: 0.9903
Classification Report:
              precision    recall  f1-score   support

           0       0.95      0.99      0.97       399
           1       1.00      0.98      0.99      1194

    accuracy                           0.99      1593
   macro avg       0.97      0.99      0.98      1593
weighted avg       0.99      0.99      0.99      1593



**Results and conclusion from the Retrained SVM model**


A mixed training and validation dataset has been used to retrain the SVM model, which has demonstrated exceptional performance in classifying test data using a balanced class weight. With an astounding precision of 99.75%, an astounding F1 score of 99.03%, and a high accuracy of 98.56%, the model demonstrated exceptional results. Additionally noteworthy is the recall rate of 98.32%, which shows how well the model can reliably identify pertinent events. These measures demonstrate the model's efficacy in reducing false positives and striking an efficient balance between precision and recall, particularly the precision and F1 score.

The classification report provides more evidence of the model's excellent discriminative power, with near-perfect scores across categories, particularly when it comes to accurately and nearly perfectly recalling the 'QUEER VOICES' category. As a result, the model's predictions are highly reliable and relevant, demonstrating how extremely well-tuned it is to the nuances of the dataset's categories.

To sum up, the retraining of the SVM model on the combined dataset has improved its predicted accuracy and reliability considerably, which makes it a great tool for high-stakes situations where fair and accurate categorization is essential. Its results indicate that combining training datasets and retraining, then carefully assessing the results, is a powerful method for obtaining good results in text classification problems. With precision and balanced recall being essential in real-world applications, this model is now ready for use.